In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
from string import punctuation
from os import listdir
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.preprocessing import StandardScaler
from keras import regularizers
from keras import optimizers
from keras.optimizers import adam

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Preprocessing

In [0]:
reviews = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Project Brian/reviews.csv")

In [0]:
resturant = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Project Brian/Restaurant_Reviews.tsv", sep='\t')

In [0]:
reviews.head()

,Review Text,Review,Recommends
0,"Friendly staff, good food and homely environme...",NaN,True
1,Well...The Food was Good___Intrerior design is...,NaN,True
2,The man who is foodie like me for him arabian ...,5.0,NaN
3,ordered pizza and they were unable to serve th...,NaN,False
4,This place is too much comfortable & food is d...,4.0,NaN


In [0]:
reviews.Review[2]

5.0

In [0]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16597 entries, 0 to 16596
Data columns (total 3 columns):
Review Text    16597 non-null object
Review         14678 non-null float64
Recommends     1918 non-null object
dtypes: float64(1), object(2)
memory usage: 389.1+ KB


In [0]:
reviews['Liked'] = -1
reviews['Liked'].loc[reviews['Review'] == 5.0] = 1
reviews['Liked'].loc[reviews['Review'] < 5.0] = 0
reviews['Liked'].loc[reviews['Recommends'] == True] = 1
reviews['Liked'].loc[reviews['Recommends'] == False] = 0

In [0]:
reviews['Review'] = reviews['Review Text']

In [0]:
reviews.head(5)

,Review Text,Review,Recommends,Liked
0,"Friendly staff, good food and homely environme...","Friendly staff, good food and homely environme...",True,1
1,Well...The Food was Good___Intrerior design is...,Well...The Food was Good___Intrerior design is...,True,1
2,The man who is foodie like me for him arabian ...,The man who is foodie like me for him arabian ...,NaN,1
3,ordered pizza and they were unable to serve th...,ordered pizza and they were unable to serve th...,False,0
4,This place is too much comfortable & food is d...,This place is too much comfortable & food is d...,NaN,0


In [0]:
editedReview = pd.DataFrame()
editedReview['Review'] = reviews['Review']
editedReview['Liked'] = reviews['Liked']
editedReview.head()

In [0]:
resturant.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [0]:
import nltk
nltk.download('stopwords')

In [0]:
stop_words = set(stopwords.words('english'))

In [0]:
punctuation
len(punctuation)

32

In [0]:
editedReview['Review'] = editedReview['Review'].str.split()

In [0]:
editedReview['ReviewTokenize'] = tokensList

In [0]:
editedReview.head()

,Review,Liked,ReviewTokenize
0,"[Friendly, staff,, good, food, and, homely, en...",1,"[friendly, staff, good, food, homely, environm..."
1,"[Well...The, Food, was, Good___Intrerior, desi...",1,"[well, food, good, intrerior, design, nice, en..."
2,"[The, man, who, is, foodie, like, me, for, him...",1,"[man, foodie, like, arabian, master, nice, pla..."
3,"[ordered, pizza, and, they, were, unable, to, ...",0,"[ordered, pizza, unable, serve, ordered, set, ..."
4,"[This, place, is, too, much, comfortable, &, f...",0,"[place, much, comfortable, food, delicious, ev..."


In [0]:
editedResturant = resturant.copy()
editedResturant.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [0]:
editedResturant['Review'] = editedResturant['Review'].str.split()

In [0]:
tokensList = []
tokens = []
for line in editedResturant['Review']:
  tokens = line
  table = str.maketrans('😜😊' + punctuation,'                                  ', '')
  result = []
  for word in tokens:
    word = word.lower()
    l = word.translate(table)
    result = result + l.split(" ")
  tokens =[word.translate(table) for word in result]
  table = str.maketrans('','', punctuation)
  tokens =[word.translate(table) for word in tokens]
  tokens = [word for word in tokens if word.isalpha()]
    # filter out stopwords
  tokens = [word for word in tokens if not word in stop_words]
  tokensList.append(tokens)
editedResturant['ReviewTokenize'] = tokensList
editedResturant.head()

,Review,Liked,ReviewTokenize
0,"[Wow..., Loved, this, place.]",1,"[wow, loved, place]"
1,"[Crust, is, not, good.]",0,"[crust, good]"
2,"[Not, tasty, and, the, texture, was, just, nas...",0,"[tasty, texture, nasty]"
3,"[Stopped, by, during, the, late, May, bank, ho...",1,"[stopped, late, may, bank, holiday, rick, stev..."
4,"[The, selection, on, the, menu, was, great, an...",1,"[selection, menu, great, prices]"


# Neural Network 

In [0]:
#gloveModel Imports
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [0]:
#Read the predefined package from research 
import gensim.downloader as api
word_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
sim = word_vectors.n_similarity(['sushi', 'shop'], ['japanese', 'restaurant'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
sim

0.7066633

In [0]:
poitive = open('/content/drive/My Drive/Colab Notebooks/Project Brian/positive-words.txt').read().split('\n')
#negative = open('/content/drive/My Drive/Colab Notebooks/Project Brian/negative-words.txt').read().split('\n')

In [0]:
sim = word_vectors.wmdistance(editedResturant['ReviewTokenize'][4], poitive)
sim

6.544278576640395

In [0]:
#convert each word into distances 
res = pd.DataFrame()
res['y'] = editedResturant['Liked']

posScore = []
poslistScore = []
for element in editedResturant['ReviewTokenize']:
  posScore.append(word_vectors.wmdistance(element, poitive))
  l = []
  for word in element:
    l.append(word_vectors.wmdistance(word, poitive))
  poslistScore.append(l)
res['posScore'] = posScore
res['posListScore'] = poslistScore
res.head()

,y,posScore,posListScore
0,1,6.142225,"[7.458121409352472, 7.446665936036574, 7.30616..."
1,0,6.468736,"[7.22366408529583, 7.391361957703441]"
2,0,6.155787,"[7.113257389235266, 7.329212824743773, 7.10858..."
3,1,6.430529,"[7.23008866450216, 7.1403864593157085, 7.41719..."
4,1,6.544279,"[7.139057824871522, 7.29461107646869, 7.253130..."


In [0]:
#convert each word into distances 
rev = pd.DataFrame()
rev['y'] = editedReview['Liked']

posScore = []
poslistScore = []
for element in editedReview['ReviewTokenize']:
  posScore.append(word_vectors.wmdistance(element, poitive))
  l = []
  for word in element:
    l.append(word_vectors.wmdistance(word, poitive))
  poslistScore.append(l)
rev['posScore'] = posScore
rev['posListScore'] = poslistScore
rev.to_csv("/content/drive/My Drive/Colab Notebooks/Project Brian/rev.csv")
rev.head()

In [0]:
res.to_csv("/content/drive/My Drive/Colab Notebooks/Project Brian/res.csv")

## Start Training the Model 

In [0]:
res = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Project Brian/res.csv')

In [0]:
res = res.replace([np.inf, -np.inf], np.nan).dropna()
res.describe()

,Unnamed: 0,y,posScore
count,999.000000,999.000000,999.000000
mean,499.859860,0.500501,6.278644
std,288.739713,0.500250,0.301358
min,0.000000,0.000000,5.342560
25%,250.500000,0.000000,6.100271
50%,500.000000,1.000000,6.255398
75%,749.500000,1.000000,6.428220
max,999.000000,1.000000,8.305947


In [0]:
y = pd.DataFrame(res['y'])
x = pd.DataFrame()
x['posScore'] = res['posScore']
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 0 to 999
Data columns (total 1 columns):
posScore    999 non-null float64
dtypes: float64(1)
memory usage: 15.6 KB


In [0]:
maxA = []
minA = []
meanA = []
d = 0
for element in res['posListScore']:
  d= d + 1
  a = element.strip('[]').split(',')
  for i in range(len(a)):
    if a == ['']:
      a = [0]
    else:
      a[i] = float(a[i].strip())
  maxA.append(max(a))
  minA.append(min(a))
  meanA.append(sum(a)/len(a))

In [0]:
x['max'] = maxA
x['min'] = minA
x['mean'] = meanA
x.describe()

,posScore,max,min,mean
count,999.000000,999.000000,999.000000,999.000000
mean,6.278644,7.434684,7.076513,7.252922
std,0.301358,0.141001,0.127778,0.087921
min,5.342560,6.945787,6.814215,6.945787
25%,6.100271,7.370653,6.995222,7.199426
50%,6.255398,7.417377,7.058258,7.254803
75%,6.428220,7.506137,7.147565,7.307521
max,8.305947,7.939488,7.545414,7.603868


In [0]:
#normalize dataset 
scalar = StandardScaler()
x = scalar.fit_transform(x)
#reviewssss2 = scalar.transform(reviewssss2)

#split training and testing data 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)


print("Training Dataset Size: ", x_train.shape)
print("Training Labels Size: ", y_train.shape)
print("Testing Dataset Size: ", x_test.shape)
print("Testing Labels Size: ", y_test.shape)

Training Dataset Size:  (669, 4)
Training Labels Size:  (669, 1)
Testing Dataset Size:  (330, 4)
Testing Labels Size:  (330, 1)


In [0]:
#set up the model architecture 
model1 = Sequential()
model1.add(Dense(10, activation='sigmoid', input_shape=(4,)))
#model1.add(Dense(40, activation='sigmoid', bias_regularizer=regularizers.l2(0.01)))
model1.add(Dense(20, activation='sigmoid'))
model1.add(Dense(1,))

model1.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 20)                220       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 291
Trainable params: 291
Non-trainable params: 0
_________________________________________________________________


In [0]:
#train the model 
op = optimizers.Adam(lr=0.01)
model1.compile(loss='mae', optimizer=op)
callbacks = []
sample = 10
history = model1.fit(x_train, y_train, batch_size=sample, epochs=10, callbacks=callbacks, verbose=1)





Epoch 1/10





669/669 [==============================] - 10s 15ms/step - loss: 0.4813
Epoch 2/10
669/669 [==============================] - 0s 322us/step - loss: 0.4907
Epoch 3/10
669/669 [==============================] - 0s 340us/step - loss: 0.5049
Epoch 4/10
669/669 [==============================] - 0s 294us/step - loss: 0.4985
Epoch 5/10
669/669 [==============================] - 0s 287us/step - loss: 0.4967
Epoch 6/10
669/669 [==============================] - 0s 289us/step - loss: 0.4960
Epoch 7/10
669/669 [==============================] - 0s 295us/step - loss: 0.4926
Epoch 8/10
669/669 [==============================] - 0s 299us/step - loss: 0.4916
Epoch 9/10
669/669 [==============================] - 0s 297us/step - loss: 0.4858
Epoch 10/10
669/669 [==============================] - 0s 286us/step - loss: 0.4765


In [0]:
#evaluate the model
mae = model1.evaluate(x_test, y_test)

print("MAE: %.4f" % mae)

330/330 [==============================] - 0s 134us/step
MAE: 0.5033


In [0]:
y_pred = model1.predict(x_test)
y_labelpred = 0
y_test1 = y_test.to_numpy()
for i in range(len(y_pred)):
  if (y_pred[i] < 0.5) and (y_test1[i] < 0.5):
    y_labelpred += 1
  elif (y_pred[i] >= 0.5) and (y_test1[i] >= 0.5):
    y_labelpred += 1
  else:
    pass
y_labelpred/len(y_pred)

0.48484848484848486

In [0]:
y_test1 = y_test.to_numpy()
y_test1

Test Output